<a href="https://colab.research.google.com/github/byznzco/NER_DataCreator_BERTurk/blob/main/NER_DataCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import math
import numpy as np
from tqdm import tqdm,trange
import re
import nltk
from keras.preprocessing.text import text_to_word_sequence


In [ ]:
df_data = pd.read_csv("/content/drive/MyDrive/ticaret-yorum.csv", delimiter=',')


In [ ]:
df_data

,category,text
0,alisveris,Altus Hırdavat Yapı Malzemeleri Drone Diye Kar...
1,alisveris,"Albay Bilgisayar Garanti Yalanı İle Yanılttı,G..."
2,alisveris,"Okyanus Petshop Ürün İade Sorunu!,Satıcıdan Gi..."
3,alisveris,Anda İhtiyaç Ve Tüketim Maddeleri Knee Tork Di...
4,alisveris,BB Kargo Dağıtım'dan Aldığım Ürün Bozuk Çıktı ...
...,...,...
431301,ulasim,Pegasus Kabin Boyu Bagajım İçin 20 Euro Aldıla...
431302,ulasim,"Pegasus Sefer Saat Değişikliği,""05.02.2020 tar..."
431303,ulasim,"THY Saygısız Müşteri Temsilcisi!,""THY'den bile..."
431304,ulasim,"KLM Havayolları Bilgisizlikleri,Kraliyet firma..."


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")


```
[{'entity_group': 'ORG',
  'score': 0.80869347,
  'word': 'Altus Hırdavat Yapı',
  'start': 0,
  'end': 19},
 {'entity_group': 'ORG',
  'score': 0.6942621,
  'word': 'Instagram',
  'start': 75,
  'end': 84}]
  ```



**read back**
```
with open('dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)
```

In [ ]:
def prepareIt(ex):
  ex = ex.replace('"', " ")
  ex = ex.replace(",", " ")
  ex = ex.replace("'", " ")
  ex = ex.replace("...", " ")
  ex = ex.replace("\n", " ")
  ex = ex.replace("\\n", " ")
  ex = ex.replace("\\t", " ")
  ex = ex.replace("\t", " ")
  ex = ex.replace("\\", " ")
  ex = ex.replace("  ", " ")
  return ex

In [ ]:
def getEntities(tokens):
  tup =()
  entities = []
  for t in tokens:
    start= t["start"]
    end = t["end"]
    ent = t["entity_group"]
    tup = (start, end, ent)
    entities.append(tup)
  return entities

In [ ]:
def labelData(data):
  tag=[]
  words = []
  pre = 0
  for i in data[1]["entities"]:
    a = data[0][i[0]:i[1]].split()
    t = pre + i[1]
    #print(pre, i[0])
    b = data[0][pre:i[0]].split()
    for j in b:
      # print("'%s': '%s'," % (j, "O"))
      tag.append("O")
      words.append(j)
      pre = i[1]
      for j in range(len(a)): 
        if j == 0:
          # print("'%s': '%s-%s'," % (a[j], "B", i[2]))
           tag.append("B-"+i[2])
           words.append(a[j])
        else: 
         # print("'%s': '%s-%s'," % (a[j], "I", i[2]))
         tag.append("I-"+i[2])
         words.append(a[j])
  #print(i[1], las)
  las = len(data[0])
  c = data[0][i[1]:las].split()
  for j in range(len(c)):
    if j == len(c) - 1:
      # print("'%s': '%s'" % (c[j], "O"))
      tag.append("O")
      words.append(c[j])
    else:
      # print("'%s': '%s'," % (c[j], "O"))
      tag.append("O")
      words.append(c[j])

  return tag, words

In [ ]:
data = ("Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country. Many people have been killed that day.",
        {"entities": [(48, 54, 'Category 1'), (77, 81, 'Category 1'), (111, 118, 'Category 2'), (150, 173, 'Category 3')]})

In [ ]:
tag,words = labelData(data)

In [ ]:
print(tag)

['O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 1', 'O', 'B-Category 2', 'O', 'B-Category 2', 'O', 'B-Category 2', 'O', 'B-Category 2', 'O', 'B-Category 2', 'O', 'B-Category 3', 'I-Category 3', 'I-Category 3', 'I-Category 3', 'O', 'B-Category 3', 'I-Category 3', 'I-Category 3', 'I-Category 3', 'O', 'B-Category 3', 'I-Category 3', 'I-Category 3', 'I-Category 3', 'O', 'B-Category 3', 'I-Category 3', 'I-Category 3', 'I-Category 3', 'O', 'B-Category 3', 'I-Category 3', 'I-Category 3', 'I-Category 3']


In [ ]:
len(splitWords(data[0]))

30

In [ ]:
t = label_list[1]["words"]

In [ ]:
df = df_data

In [ ]:
# take random 2000 data
from sklearn.utils import shuffle
df_data = shuffle(df_data)
df_data
df_data = df_data.sample(n=5000)
df_data

In [ ]:
from tqdm.auto import tqdm
label_list = []
no_label_data = []
no_label = 1

c = 0
for d in tqdm(df_data["text"]):
  c +=1
  d = prepareIt(d)
  tokens = ner(d)
  d = d.lower()

  entities = getEntities(tokens)
  
  m = (d,{"entities":entities})
  if not m[1]["entities"]:
    print("Error no labeled data ", no_label)
    n = len(words) 
    a = 'O'
    tag= [a]*n
    a = {
      "sentence#": c,
      "words": words,
      "tag": tag
    }
    no_label_data.append(a)
    no_label += 1
  else: 
    tag,words = labelData(m)
    a = {
        "sentence#": c,
        "words": words,
        "tag": tag
      }
  label_list.append(a)
  

  0%|          | 0/5000 [00:00<?, ?it/s]

Error no labeled data  1
Error no labeled data  2
Error no labeled data  3
Error no labeled data  4
Error no labeled data  5
Error no labeled data  6
Error no labeled data  7
Error no labeled data  8
Error no labeled data  9
Error no labeled data  10
Error no labeled data  11
Error no labeled data  12
Error no labeled data  13
Error no labeled data  14
Error no labeled data  15
Error no labeled data  16
Error no labeled data  17
Error no labeled data  18
Error no labeled data  19
Error no labeled data  20
Error no labeled data  21
Error no labeled data  22
Error no labeled data  23
Error no labeled data  24
Error no labeled data  25
Error no labeled data  26
Error no labeled data  27
Error no labeled data  28
Error no labeled data  29
Error no labeled data  30
Error no labeled data  31
Error no labeled data  32
Error no labeled data  33
Error no labeled data  34
Error no labeled data  35
Error no labeled data  36
Error no labeled data  37
Error no labeled data  38
Error no labeled data

In [ ]:
label_list

KeyboardInterrupt: ignored

In [ ]:
id_list=[]
word_list=[]
tag_list=[]

for l in label_list:
  id = l["sentence#"]
  id_list.append(id)

  for w in l["words"]:
    word_list.append(w)
  for i in range(len(l["words"])-1):
    id_list.append("")
  
  for t in l["tag"]:
    tag_list.append(t)

In [ ]:
print(len(id_list), len(word_list),len(tag_list))

313399 313399 313399


In [ ]:
df_test = pd.DataFrame()
df_test['Sentence#']  = id_list
df_test['Words']  = word_list
df_test['Tag']  = tag_list

In [ ]:
df_test.to_csv("labels.csv")

In [ ]:
import csv
 
# opening the csv file in 'w' mode
file = open('g4g.csv', 'w', newline ='')
 
with file:
    # identifying header 
    header = ['sentence#', 'words', 'tag']
    writer = csv.DictWriter(file, fieldnames = header)
     
    # writing data row-wise into the csv file
    writer.writeheader()
    for l in label_list:
      sentence_id = l["sentence#"]
      words = l["words"]
      tag = l["tag"]
      writer.writerow({'sentence#' : sentence_id,
                     'words': words,
                     'tag': tag})




In [ ]:
import pandas as pd

df = pd.read_csv('g4g.csv')

print(df) 

FileNotFoundError: ignored

```
@software{stefan_schweter_2020_3770924,
  author       = {Stefan Schweter},
  title        = {BERTurk - BERT models for Turkish},
  month        = apr,
  year         = 2020,
  publisher    = {Zenodo},
  version      = {1.0.0},
  doi          = {10.5281/zenodo.3770924},
  url          = {https://doi.org/10.5281/zenodo.3770924}
}
```

```
@software{Taner Akdeniz,
  author       = {Taner Akdeniz},
  title        = {akdeniz27/bert-base-turkish-cased-ner},
  month        = sep,
  year         = 2021,
  url          = {https://huggingface.co/akdeniz27/bert-base-turkish-cased-ner}
}
```
[NER MODEL](https://huggingface.co/busecarik/berturk-sunlp-ner-turkish)